In [1]:
import sys
from tqdm import tqdm
import pdb
import torch
import logging
import numpy as np
import dgl
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import coo_matrix, csr_matrix

In [2]:
data='TaoBao'
train_path = './datasets/' + data + '/train.txt'
val_path = './datasets/' + data + '/val.txt'
test_path = './datasets/' + data + '/test.txt'
category_path = './datasets/' + data + '/item_category.txt'



In [3]:
def read_category(path):
        num = 0
        dic = {}
        with open(path, 'r') as f:
            lines = f.readlines()
            for line in tqdm(lines):
                line = line.strip().split(',')
                item = int(line[0])
                category = int(line[1])
                if category not in dic.values():
                    num += 1
                dic[item] = category
        return dic, num
category_dic, category_num = read_category(category_path)


100%|██████████| 136710/136710 [00:07<00:00, 19324.58it/s]


In [ ]:
def read_train_graph(path):
        historical_dict = []
        train_data = []
        with open(path, 'r') as f:
            lines = f.readlines()
            for line in tqdm(lines):
                line = line.strip().split(',')
                user = int(line[0])
                item = int(line[1])
                if user in historical_dict:
                    historical_dict[user].add(item)
                else:
                    historical_dict[user] = set([item])
        return historical_dict


In [34]:
def read_train_graph(path):
        historical_dict = []
        train_data = []
        with open(path, 'r') as f:
            lines = f.readlines()
            for line in tqdm(lines):
                line = line.strip().split(',')
                user = int(line[0])
                item = int(line[1])
                train_data.append([user, item])

                if user in historical_dict:
                    historical_dict[user].add(item)
                else:
                    historical_dict[user] = set([item])

        train_data = torch.tensor(train_data)
        user_number = max(user_number, train_data[:, 0].max() + 1)
        item_number = max(item_number, train_data[:, 1].max() + 1)

        graph_data = {
            ('user', 'rate', 'item'): (train_data[:, 0].long(), train_data[:, 1].long()),
            ('item', 'rated by', 'user'): (train_data[:, 1].long(), train_data[:, 0].long())
        }
        graph = dgl.heterograph(graph_data)
        category_tensor = torch.tensor(list(category_dic.values()), dtype = torch.long).unsqueeze(1)
        graph.ndata['category'] = {'item': category_tensor, 'user': torch.zeros(user_number, 1) - 1}

        return graph

In [35]:
historical_dict = []
train_data = []
with open(train_path, 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
            line = line.strip().split(',')
            user = int(line[0])
            item = int(line[1])
            train_data.append([user, item])

            if user in historical_dict:
                historical_dict[user].append(category_dic[item])
            else:
                historical_dict[user] =[category_dic[item]]
    


100%|██████████| 2571752/2571752 [00:05<00:00, 468459.10it/s]


In [93]:
import math
import random
user_select=[]
maxlen=20
cout=0
for line in tqdm(historical_dict.values()):
    select=[]
    unique_elements = list(set(line))
    element_counts = [line.count(element) for element in unique_elements]
    element_indices = {}
    for index, element in enumerate(line):
        if element in element_indices:
            element_indices[element].append(index)
        else:
            element_indices[element] = [index]
    avg=math.ceil(sum(element_counts)/len(element_counts))
    
    sorted_indices = sorted(range(len(element_counts)), key=lambda i: element_counts[i], reverse=True)
    
    for i in sorted_indices:
        my_list=element_indices[unique_elements[i]]
        if int(len(my_list)/2)>avg:
            
            random_elements = random.sample(my_list,int(len(my_list)/2))
        else:
            random_elements=random.sample(my_list, k=min(avg,len(my_list)))
        select=select+random_elements
    if len(select)>=maxlen:
        select=select[0:maxlen]
    else:
        select=select+select[0:maxlen-len(select)]
        cout=cout+1
    user_select.append(select)
    


100%|██████████| 82633/82633 [00:04<00:00, 19986.96it/s]


In [94]:
cout

43762

In [70]:
print(line)
print(unique_elements)
print(element_counts)
print(top_5_elements)
print(top_5_indices)

[0, 1, 4, 5, 4, 5, 1, 7, 5, 8, 11, 5, 5, 5, 5, 12]
[0, 1, 4, 5, 7, 8, 11, 12]
[1, 2, 2, 7, 1, 1, 1, 1]
[7, 2, 2, 1]
[3, 1, 2, 0]


In [71]:
print(element_indices)

{0: [0], 1: [1, 6], 4: [2, 4], 5: [3, 5, 8, 11, 12, 13, 14], 7: [7], 8: [9], 11: [10], 12: [15]}


In [72]:
print(user_select)

[[3, 8, 1, 6, 2, 4, 0]]


In [79]:
a=[1,2,3,4]
print(a[-1:])

[4]
